In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown


In [3]:
import zipfile
zip_ref = zipfile.ZipFile("/content/dogs-vs-cats.zip",'r')
zip_ref.extractall("/content")
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten

In [5]:
from keras.applications.vgg16 import VGG16

In [6]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (150,150,3)
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [8]:
conv_base.trainable = False

# DATA AUGMENTATION

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array,load_img

In [10]:
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
    "/content/train",
    target_size = (150,150),
    batch_size = batch_size,
    class_mode = 'binary'
)
validation_gen = test_datagen.flow_from_directory(
    "/content/test",
    target_size = (150,150),
    batch_size = batch_size,
    class_mode = "binary"
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [11]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics =['accuracy'])

In [12]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = validation_gen
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
418/625 ━━━━━━━━━━━━━━━━━━━━ 29:36 9s/step - accuracy: 0.8078 - loss: 0.4734